# Variable Types

Variables types are important factors of Featuretools. It is important that datasets have appropriately defined variables because this will allow the right primitives to be used to generate new features. 

To understand the different variable types, let's first look at a graph of the variables:

In [ ]:
from featuretools.variable_types.utils import graph_variable_types
graph_variable_types()

As we can see, there are multiple variable types and some have subclassed variable types. For example, ZIPCode is variable type that is child of Categorical type which is a subtype of Discrete type. 

We can also get all the variable types as a DataFrame.

In [ ]:
from featuretools.variable_types.utils import list_variable_types
list_variable_types()